In [8]:
import numpy as np
from keras import backend as K
from keras.layers import Input, Embedding, merge
import keras.layers
from keras.regularizers import l2, l1
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.models import Sequential, Model
from keras.optimizers import SGD, RMSprop, Adam
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import pydotplus as pydot 
import graphviz
from keras.utils import plot_model


In [9]:
import pandas as pd

In [10]:
csvtoload = 'animeedascores.csv'

Data is loaded as ratings temp, then transformed into ratings 1,2,3 then fed into a neural Net. THe data istransformed differently for temp 4. 
1st removes all 0's. 
2nd is limited only to shows that are completed.
3rd uses a some numerical transformations to reward intend to see, penalize on hold, and greatly penalize dropped shows. 
4th doesn't modify the data, but will be fed into a neural network that will attempt to embed the status with n hidden factors, which will be appended to the anime's status. 

In [11]:
ratingstemp = pd.read_csv(csvtoload) # This is a subset of reviews with all 0's dropped. 
ratingstemp['userid']=ratingstemp['index']
ratingstemp['score']=ratingstemp['animescore']

In [12]:
ratings_1=ratingstemp[(ratingstemp['score']!=0)] # this represents only shows that have a non zero score

In [13]:
ratings_2=ratingstemp[ratingstemp['status']=='COMPLETED'] # this represents only shows that viewer claims to have completed.

In [14]:
ratings_3= ratingstemp.copy()# this involves some more advanced modifications


In [15]:
ratings_3.head()

,Unnamed: 0,Unnamed: 0.1,animeid,animescore,index,name,status,unkey,userid,score
0,0,0,32878,0,palelion,palelion,CONSUMING,32878palelion,palelion,0
1,1,1,31964,0,palelion,palelion,CONSUMING,31964palelion,palelion,0
2,2,2,13367,0,palelion,palelion,CONSUMING,13367palelion,palelion,0
3,3,3,189,0,palelion,palelion,CONSUMING,189palelion,palelion,0
4,4,4,330,0,palelion,palelion,CONSUMING,330palelion,palelion,0


In [16]:
scoreadj = {'COMPLETED':0,'CONSUMING':0,'DROPPED':-2,'ONHOLD':-1,'BACKLOG':-.5}

ratings_3['score_adj']= ratings_3.status.map(scoreadj)

In [17]:
ratings_3['score']=ratings_3['score']+ratings_3['score_adj']

In [18]:
#ratings_3.tail()

In [19]:
ratings_4=ratingstemp.copy()# this is the same as ratings 3, but with imputed scores for 0's.

In [20]:
ratings_4_1 = ratings_4[ratings_4['score']==0]
ratings_4_2 = ratings_4[ratings_4['score']!=0]
scoreimpute = {'COMPLETED':8,'CONSUMING':7,'DROPPED':3,'ONHOLD':5,'BACKLOG':6}

In [21]:
ratings_4_1['score']=ratings_4_1.status.map(scoreimpute)

/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [22]:
#ratings_4_1.tail()

In [23]:
ratings_4.loc[ratings_4_1.index,'score']= ratings_4_1['score']

In [24]:
#ratings_4_1['score']

In [25]:
#ratings_4.head()

In [26]:
#ratings_4=ratings_4[ratings_4['status']!='BACKLOG']
#ratings_4['status'].value_counts()

In [27]:
ratings_5= ratingstemp.copy() # this will be unmodified, but the neural network will have a different architecture

In [28]:
def prep_for_keras1(df):
    users = df.userid.unique()
    animes = df.animeid.unique()
    userid2idx = {o:i for i,o in enumerate(users)} # turn the user strings into a numerical value. 
    animeid2idx = {o:i for i,o in enumerate(animes)} # remove missing anime numbers and re-order
    df['anime_id'] = df.animeid.apply(lambda x: animeid2idx[x]) #the above code set up mapping, this applies it. 
    df.userid = df.userid.apply(lambda x: userid2idx[x])# the above code set up mapping, this applies it.
    n_users = df.userid.nunique()
    n_animes = df.anime_id.nunique()
    return df,n_users, n_animes,userid2idx,animeid2idx

In [29]:
ratings1, n_users1,n_animes1,useridnn1,animeidnn1 = prep_for_keras1(ratings_1)

/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/nerd/Data/Conda/lib/python3.6/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [30]:
n_factors = 40

In [31]:
msk1 = np.random.rand(len(ratings1)) < 0.8 #training vs test set ratio
trn1 = ratings1[msk1]
val1 = ratings1[~msk1]

In [32]:
user_in = Input(shape=(1,), dtype='int64', name='user_in')
u = Embedding(n_users1, n_factors, input_length=1, embeddings_regularizer=l2(1e-4))(user_in)
anime_in = Input(shape=(1,), dtype='int64', name='anime_in')
m = Embedding(n_animes1, n_factors, input_length=1, embeddings_regularizer=l2(1e-4))(anime_in)

In [26]:
x = keras.layers.dot([u, m], axes=2)
x = Flatten()(x)
model = Model([user_in, anime_in], x)
model.compile(Adam(0.001), loss='mae')

In [27]:
model.lr=.5

In [28]:
model.fit([trn1.userid, trn1.anime_id], trn1.score, batch_size=5120, epochs=1, 
          validation_data=([val1.userid, val1.anime_id], val1.score))

Train on 3795465 samples, validate on 947488 samples
Epoch 1/1
3795465/3795465 [==============================] - 9s - loss: 7.7553 - val_loss: 7.7524


In [45]:
def embedding_input_anime(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_Anime_Hidden_Factors')(inp)
def embedding_input_user(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_User_Hidden_Factors')(inp)
def embedding_input_status(name, n_in, n_out, reg):
    inp = Input(shape=(1,), dtype='int64', name=name)
    return inp, Embedding(n_in, n_out, input_length=1, W_regularizer=l2(reg),name='Embed_Status_Hidden_Factors')(inp)

In [34]:
user_in1, u1 = embedding_input_user('user_id_in', n_users1, n_factors, 1e-4)
anime_in1, a1 = embedding_input_anime('anime_id_in', n_animes1, n_factors, 1e-4)

/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(74849, 40, input_length=1, name="Embed_User_Hidden_Factors", embeddings_regularizer=<keras.reg...)`
  
/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(10412, 40, input_length=1, name="Embed_Anime_Hidden_Factors", embeddings_regularizer=<keras.reg...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [37]:
x = merge([u1, a1], mode='concat', name='All_Factors_on_one_layer')
x = Flatten()(x)
x = Dropout(0.3, name='Prevent_overfit')(x)
x = Dense(70, activation='relu',name='Random_HF_Interactions')(x)
x = Dropout(0.75,name='Prevent_overfit2')(x)
x = Dense(1,name='Final_Interactions')(x)
nn1 = Model([user_in1, anime_in1], x)
nn1.compile(Adam(0.001), loss='mse')

/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
/home/nerd/Data/Conda/lib/python3.6/site-packages/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [32]:
nn1.fit([trn1.userid, trn1.anime_id], trn1.score, batch_size=5120, epochs=1, 
          validation_data=([val1.userid, val1.anime_id], val1.score))

Train on 3795465 samples, validate on 947488 samples
Epoch 1/1
3795465/3795465 [==============================] - 8s - loss: 11.8896 - val_loss: 2.4877


In [33]:
nn1.lr=.5

In [34]:
nn1.fit([trn.userid, trn.anime_id], trn.score, batch_size=5120, epochs=6, 
          validation_data=([val.userid, val.anime_id], val.score))

NameError: name 'trn' is not defined

In [ ]:
nn1.lr=.01

In [ ]:
nn1.fit([trn.userid, trn.anime_id], trn.score, batch_size=5120, epochs=18, 
          validation_data=([val.userid, val.anime_id], val.score))

In [38]:
plot_model(nn1, to_file='nn1_1.png')

In [69]:
nn1.save_weights('nn1weights.h5')

In [37]:
ratings2, n_users2,n_animes2,useridnn2,animeidnn2 = prep_for_keras1(ratings_2)

/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/nerd/Data/Conda/lib/python3.6/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [40]:
msk2 = np.random.rand(len(ratings2)) < 0.8 #training vs test set ratio
trn2 = ratings2[msk2]
val2 = ratings2[~msk2]

In [41]:
user_in2, u2 = embedding_input('user_in', n_users2, n_factors, 1e-4)
anim_in2, a2 = embedding_input('anime_in', n_animes2, n_factors, 1e-4)

/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(78252, 40, input_length=1, embeddings_regularizer=<keras.reg...)`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(10384, 40, input_length=1, embeddings_regularizer=<keras.reg...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [43]:
x = merge([u2, a2], mode='concat')
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn2 = Model([user_in2, anim_in2], x)
nn2.compile(Adam(0.001), loss='mae')

/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
/home/nerd/Data/Conda/lib/python3.6/site-packages/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [44]:
nn2.fit([trn2.userid, trn2.anime_id], trn2.score, batch_size=5120, epochs=1, 
          validation_data=([val2.userid, val2.anime_id], val2.score))

Train on 4619245 samples, validate on 1156121 samples
Epoch 1/1
4619245/4619245 [==============================] - 11s - loss: 2.6253 - val_loss: 1.5751


In [70]:
nn2.save_weights('nn2weights.h5')

In [46]:
ratings3, n_users3,n_animes3,useridnn3,animeidnn3 = prep_for_keras1(ratings_3)

In [47]:
msk3 = np.random.rand(len(ratings3)) < 0.8 #training vs test set ratio
trn3 = ratings3[msk3]
val3 = ratings3[~msk3]

In [48]:
user_in3, u3 = embedding_input('user_in', n_users3, n_factors, 1e-4)
anim_in3, a3 = embedding_input('anime_in', n_animes3, n_factors, 1e-4)

/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(90516, 40, input_length=1, embeddings_regularizer=<keras.reg...)`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(11907, 40, input_length=1, embeddings_regularizer=<keras.reg...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [49]:
x = merge([u3, a3], mode='concat')
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn3 = Model([user_in3, anim_in3], x)
nn3.compile(Adam(0.001), loss='mae')

/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
/home/nerd/Data/Conda/lib/python3.6/site-packages/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [50]:
nn3.fit([trn3.userid, trn3.anime_id], trn3.score, batch_size=5120, epochs=1, 
          validation_data=([val3.userid, val3.anime_id], val3.score))

Train on 7193177 samples, validate on 1795890 samples
Epoch 1/1
7193177/7193177 [==============================] - 19s - loss: 2.9119 - val_loss: 2.4947


In [71]:
nn3.save_weights('nn3weights.h5')

In [51]:
ratings4, n_users4,n_animes4,useridnn4,animeidnn4 = prep_for_keras1(ratings_4)

In [52]:
msk4 = np.random.rand(len(ratings4)) < 0.8 #training vs test set ratio
trn4 = ratings4[msk4]
val4 = ratings4[~msk4]

In [53]:
user_in4, u4 = embedding_input('user_in', n_users4, n_factors, 1e-4)
anim_in4, a4 = embedding_input('anime_in', n_animes4, n_factors, 1e-4)

/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(90516, 40, input_length=1, embeddings_regularizer=<keras.reg...)`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(11907, 40, input_length=1, embeddings_regularizer=<keras.reg...)`
  This is separate from the ipykernel package so we can avoid doing imports until


In [56]:
x = merge([u4, a4], mode='concat')
x = Flatten()(x)
x = Dropout(0.3)(x)
x = Dense(70, activation='relu')(x)
x = Dropout(0.75)(x)
x = Dense(1)(x)
nn4 = Model([user_in4, anim_in4], x)
nn4.compile(Adam(0.001), loss='mae')

/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
/home/nerd/Data/Conda/lib/python3.6/site-packages/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [57]:
nn4.fit([trn4.userid, trn4.anime_id], trn4.score, batch_size=5120, epochs=1, 
          validation_data=([val4.userid, val4.anime_id], val4.score))

Train on 7192052 samples, validate on 1797015 samples
Epoch 1/1
7192052/7192052 [==============================] - 18s - loss: 2.3500 - val_loss: 1.2274


In [72]:
nn4.save_weights('nn4weights.h5')

In [39]:
ratings5, n_users5,n_animes5,useridnn5,animeidnn5 = prep_for_keras1(ratings_5)

In [40]:
def prep_for_keras2(df):
    status = df.status.unique()
    status2idx = {o:i for i,o in enumerate(status)} # turn the watched status into a vector 
    df['status_id'] = df.status.apply(lambda x: status2idx[x]) #the above code set up mapping, this applies it. 
    n_status = df.status.nunique()
    return df,n_status,status2idx

In [41]:
ratings5, n_status, status2idx = prep_for_keras2(ratings5)

In [42]:
n_status_factors = 6

In [43]:
msk5 = np.random.rand(len(ratings5)) < 0.8 #training vs test set ratio
trn5 = ratings5[msk5]
val5 = ratings5[~msk5]

In [46]:
user_in5, u5 = embedding_input_user('user_id_in', n_users5, n_factors, 1e-4)
anime_in5, a5 = embedding_input_anime('anime_id_in', n_animes5, n_factors, 1e-4)
status_in5, s5=embedding_input_status('status_id_in', n_status, n_status_factors, 1e-4)

/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(90516, 40, input_length=1, name="Embed_User_Hidden_Factors", embeddings_regularizer=<keras.reg...)`
  
/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(11907, 40, input_length=1, name="Embed_Anime_Hidden_Factors", embeddings_regularizer=<keras.reg...)`
  This is separate from the ipykernel package so we can avoid doing imports until
/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(5, 6, input_length=1, name="Embed_Status_Hidden_Factors", embeddings_regularizer=<keras.reg...)`
  if __name__ == '__main__':


In [47]:
x = merge([u5, a5,s5], mode='concat',name='All_Factors_on_one_layer')
x = Flatten()(x)
x = Dropout(0.3,name='Prevent_Overfit')(x)
x = Dense(70, activation='relu',name='Random_HF_Interactions')(x)
x = Dropout(0.75,name='Prevent_Overfit2')(x)
x = Dense(1,name='Final_Interactions')(x)
nn5 = Model([user_in5, anime_in5,status_in5], x)
nn5.compile(Adam(0.001), loss='mae')

/home/nerd/Data/Conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  """Entry point for launching an IPython kernel.
/home/nerd/Data/Conda/lib/python3.6/site-packages/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


In [65]:
nn5.fit([trn5.userid, trn5.anime_id,trn5.status_id], trn5.score, batch_size=5120, epochs=1, 
          validation_data=([val5.userid, val5.anime_id,val5.status_id], val5.score))

Train on 7191749 samples, validate on 1797318 samples
Epoch 1/1
7191749/7191749 [==============================] - 20s - loss: 2.1078 - val_loss: 1.4189


In [48]:
plot_model(nn5, to_file='nn5_1.png')

In [73]:
nn5.save_weights('nn5weights.h5')

In [ ]:
ratings5.tail()

In [ ]:
ub = create_bias(user_in, n_users,1e-3)
ab = create_bias(anime_in, n_animes,1e-2)
user_in, u = embedding_input('user_in', n_users, n_factors, 1e-4)
anime_in, a = embedding_input('anime_in', n_animes, n_factors, 1e-4)

In [ ]:
x = keras.layers.dot([u, m], axes=2)
x = Flatten()(x)
model = Model([user_in, anime_in], x)
model.compile(Adam(0.001), loss='mae')

In [ ]:
x = merge([u, a], mode='dot')
x = Flatten()(x)
x = merge([x, ub], mode='sum')
x = merge([x, ab], mode='sum')
model = Model([user_in, anime_in], x)
model.compile(Adam(0.001), loss='mse')

In [ ]:
x = merge([u, a], mode='dot')
x = Flatten()(x)
x = merge([x, ub], mode='sum')
x = merge([x, ab], mode='sum')
dot_product_svd = Model([user_in, anime_in], x)
dot_product_svd.compile(Adam(0.001), loss='mae')